In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk_certify_label import run
from src.data import get_graph
from src import globals
globals.init({})

In [3]:
seed = 2

data_params = dict(
    dataset = "cba",
    learning_setting = "transductive", # or "transdructive"
    specification = dict(
        classes = 2,
        n_trn_labeled = 10, 
        n_trn_unlabeled = 0,
        n_val = 10,
        n_test = 180,
        sigma = 1,
        avg_within_class_degree = 1.58 * 2,
        avg_between_class_degree = 0.37 * 2,
        K = 1.5,
        m = 2,
        seed = 2 # used to generate the dataset & data split
    )
)

model_params = dict(
    label = "APPNP",
    model = "APPNP",
    normalization = "sym_normalization",
    alpha = 0,
    iteration = 10,
    depth = 1,
    #regularizer = 1e-8
    regularizer = 0.5,
    pred_method = "svm",
    activation = "relu",
    solver = "qplayer",
    alpha_tol = 1e-4,
    bias = False,
)

certificate_params = dict(
    n_adversarial = 1, # number adversarial nodes
    perturbation_model = "linf",
    delta = 0.01 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "info"
)  

other_params = dict(
    device = "cpu",
    dtype = torch.float64,
    allow_tf32 = False,
    path_gurobi_license = "/ceph/ssd/staff/gosl/app/gurobi.lic"
)

In [5]:
model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 0.075,
    pred_method = "svm",
    activation = "relu",
    solver = "qplayer",
    alpha_tol = 1e-4,
    bias = False,
)

In [7]:
model_params = dict(
    label = "SGC",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 10,
    pred_method = "svm",
    activation = "linear",
    solver = "qplayer",
    alpha_tol = 1e-4,
    bias = False,
)

In [8]:
data_params = dict(
    dataset = "cora_ml_cont_auto",    
    learning_setting = "transductive", # or "transdructive"
    specification = dict(
        n_per_class = 5,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
        seed = 2,
    )
)

In [4]:
n_edges_l = []
for seed in range(5):
    data_params["specification"]["seed"] = seed
    X, A, y, mu, p, q = get_graph(data_params, sort=True)
    n_edges_l.append(A.sum() / 2)
    print(f"{A.shape[0]}")
print(f"n_edges: {np.mean(n_edges_l)} +- {np.std(n_edges_l)}")

2024-09-28 18:32:00 (INFO): CBA(m=2, p=3.16000, q=0.74000)


2024-09-28 18:32:00 (INFO): CBA(m=2, p=3.16000, q=0.74000)
2024-09-28 18:32:00 (INFO): CBA(m=2, p=3.16000, q=0.74000)
2024-09-28 18:32:00 (INFO): CBA(m=2, p=3.16000, q=0.74000)
2024-09-28 18:32:00 (INFO): CBA(m=2, p=3.16000, q=0.74000)


200
200
200
200
200
n_edges: 389.0 +- 2.6076809620810595


In [9]:
# no big-M, no continous y
certificate_params["delta"] = 0.1
certificate_params["LogToConsole"] = 0
certificate_params["OutputFlag"] = 1
result = run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-10-01 00:31:48 (INFO): Starting experiment exp_ntk_certify_label with configuration:
2024-10-01 00:31:48 (INFO): data_params: {'dataset': 'cora_ml_cont_auto', 'learning_setting': 'transductive', 'specification': {'n_per_class': 5, 'fraction_test': 0.1, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True, 'seed': 2}}
2024-10-01 00:31:48 (INFO): model_params: {'label': 'APPNP', 'model': 'APPNP', 'normalization': 'sym_normalization', 'alpha': 0, 'iteration': 10, 'depth': 1, 'regularizer': 0.5, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer', 'alpha_tol': 0.0001, 'bias': False}
2024-10-01 00:31:48 (INFO): certification_params: {'n_adversarial': 1, 'perturbation_model': 'linf', 'delta': 0.1, 'LogToConsole': 0, 'OutputFlag': 1}


2024-10-01 00:31:48 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-10-01 00:31:48 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 'path_gurobi_license': '/ceph/ssd/staff/gosl/app/gurobi.lic'}
2024-10-01 00:31:48 (INFO): seed: 2
2024-10-01 00:31:48 (INFO): number of samples
 - labeled: 35 
 - val: 35 
 - test: 304 
 - unlabeled: 2621
2024-10-01 00:31:48 (INFO): Nr. of edges: 8158.0
2024-10-01 00:31:48 (INFO): Nr. of nodes: 2995


AssertionError: qplayer only for binary class

In [16]:
# including big-M, but not continious y
certificate_params["delta"] = 0.1
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
result = run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-08-22 15:46:06 (INFO): Starting experiment exp_ntk_certify_label with configuration:
2024-08-22 15:46:06 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'transductive', 'specification': {'classes': 2, 'n_trn_labeled': 20, 'n_trn_unlabeled': 0, 'n_val': 20, 'n_test': 5, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 4}}
2024-08-22 15:46:06 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer', 'alpha_tol': 0.0001, 'bias': False}
2024-08-22 15:46:06 (INFO): certification_params: {'n_adversarial': 1, 'perturbation_model': 'linf', 'delta': 0.1, 'method': 'XXT', 'attack_nodes': 'test', 'LogToConsole': 1, 'OutputFlag': 1}
2024-08-22 15:46:06 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-08-22 15:46:06 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 

CSBM mu:
[0.4330127 0.4330127 0.4330127]
34 alphas found: ['0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.0175', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.0449', '0.0000', '0.1000', '0.1000', '0.1000', '0.0107']
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


/ceph/ssd/staff/gosl/src/ntk-robust/src/utils.py:1023: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  m.addConstr((z_*ntk_labeled)@v_ones - u + v == 1, "eq_constraint")
2024-08-22 15:46:06 (INFO): Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


M_u: avg 0.61, max 2.67
M_v: avg 2.55, max 4.67
Set parameter BestObjStop to value 0


2024-08-22 15:46:06 (INFO): Set parameter BestObjStop to value 0


...

Best objective 4.609539906640e-01, best bound 4.609539906640e-01, gap 0.0000%


2024-08-22 15:47:09 (INFO): Best objective 4.609539906640e-01, best bound 4.609539906640e-01, gap 0.0000%


2024-08-22 15:47:09 (INFO): 


User-callback calls 4776, time in user-callback 0.01 sec


2024-08-22 15:47:09 (INFO): User-callback calls 4776, time in user-callback 0.01 sec
2024-08-22 15:47:09 (INFO): Optimization status: 2
2024-08-22 15:47:09 (INFO): Original 0.76366, Opt objective 0.46095
2024-08-22 15:47:09 (INFO): Robust count 8 out of 8
2024-08-22 15:47:09 (INFO): Certified accuracy (poisoning): 1.0


In [6]:
# big-M + integrality focus = 0
certificate_params["delta"] = 0.1
certificate_params["LogToConsole"] = 1
certificate_params["OutputFlag"] = 1
result = run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-08-22 16:05:32 (INFO): Starting experiment exp_ntk_certify_label with configuration:
2024-08-22 16:05:32 (INFO): data_params: {'dataset': 'csbm', 'learning_setting': 'transductive', 'specification': {'classes': 2, 'n_trn_labeled': 20, 'n_trn_unlabeled': 0, 'n_val': 20, 'n_test': 5, 'sigma': 1, 'avg_within_class_degree': 3.16, 'avg_between_class_degree': 0.74, 'K': 1.5, 'seed': 4}}
2024-08-22 16:05:32 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'activation': 'relu', 'solver': 'qplayer', 'alpha_tol': 0.0001, 'bias': False}
2024-08-22 16:05:32 (INFO): certification_params: {'n_adversarial': 1, 'perturbation_model': 'linf', 'delta': 0.1, 'method': 'XXT', 'attack_nodes': 'test', 'LogToConsole': 1, 'OutputFlag': 1}
2024-08-22 16:05:32 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-08-22 16:05:32 (INFO): other_params: {'device': 'cpu', 'dtype': torch.float64, 'allow_tf32': False, 

2024-08-22 16:05:32 (INFO): CSBM(p=0.14364, q=0.03364)
2024-08-22 16:05:32 (INFO): Delta: 0.1
2024-08-22 16:05:32 (INFO): Test accuracy: 1.0
2024-08-22 16:05:32 (INFO): Train accuracy: 0.9459459185600281


CSBM mu:
[0.4330127 0.4330127 0.4330127]
34 alphas found: ['0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.0175', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '0.1000', '-0.0000', '0.1000', '0.1000', '0.0449', '0.0000', '0.1000', '0.1000', '0.1000', '0.0107']
Set parameter Username


2024-08-22 16:05:32 (INFO): Set parameter Username


Academic license - for non-commercial use only - expires 2025-03-26


...

Best objective 4.609539906640e-01, best bound 4.609539906640e-01, gap 0.0000%


2024-08-22 16:06:45 (INFO): Best objective 4.609539906640e-01, best bound 4.609539906640e-01, gap 0.0000%


2024-08-22 16:06:45 (INFO): 


User-callback calls 8187, time in user-callback 0.01 sec


2024-08-22 16:06:45 (INFO): User-callback calls 8187, time in user-callback 0.01 sec
2024-08-22 16:06:45 (INFO): Optimization status: 2
2024-08-22 16:06:45 (INFO): Original 0.76366, Opt objective 0.46095
2024-08-22 16:06:45 (INFO): Robust count 8 out of 8
2024-08-22 16:06:45 (INFO): Certified accuracy (poisoning): 1.0
